<a href="https://colab.research.google.com/github/prithwis/parashar21/blob/main/P21_45_Cast_Load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](http://1.bp.blogspot.com/_5QlanosqxiQ/SXLOLTu7S7I/AAAAAAAAAm8/0r6C7lCxsic/S1600-R/p21-header-003.png) <br>


---



[Prithwis Mukerjee](http://www.yantrajaal.com) ||
Website - [Parashar21](https://parashar21.blogspot.com)<br>

Copyright (c) 2022, Prithwis Mukerjee
All rights reserved.

This source code is licensed under the GNU GPL v3.0 -style license found in the
LICENSE file in the root directory of this source tree. 

#Create Charts and store in MongoDB

#Setup

##Install Swiss Ephemeris <br>

https://pypi.org/project/pyswisseph/  - Python Package Documentation <br>
https://www.astro.com/swisseph/swephinfo_e.htm <br>
https://www.astro.com/swisseph/swephprg.htm - Programmer Documentation <br>
https://pythonhosted.org/pyswisseph/swisseph-module.html

In [1]:
!pip -qq install pyswisseph                                 # https://stackoverflow.com/questions/64277506/pip-install-options-unclear
#!pip -qq install python_string_utils
#!pip -qq install python-docx                                #https://python-docx.readthedocs.io/en/latest/
#!pip install pyswisseph==2.10.2.0.dev1

# required for accessing Atlas servers 
!/usr/bin/python3 -m pip install "pymongo[srv]"
!apt install mongodb > /dev/null

     |████████████████████████████████| 521 kB 5.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 269 kB 5.0 MB/s 




In [2]:
from pymongo import MongoClient
import pandas as pd
import dateutil
#import string_utils
import json

In [3]:
#Load p21 modules from github
!wget -O p21.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21.py                  # all global variables
!wget -O p21swe.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21swe.py            # Swiss Ephemeris functions
!wget -O p21utils.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21utils.py        # large number of utility functions



In [4]:
# ---------------------------
import p21
import p21swe
import p21utils
# ------------------------------------------------
# required for multiple imports of the same module
# ------------------------------------------------
import importlib
importlib.reload(p21)
importlib.reload(p21swe)
importlib.reload(p21utils)

imported p21
p21swe imported
p21utils imported
imported p21
p21swe imported
p21utils imported


<module 'p21utils' from '/content/p21utils.py'>

In [5]:
#Download Swiss Ephemeris Files 
!wget -q http://www.astro.com/ftp/swisseph/ephe/seas_18.se1
!wget -q http://www.astro.com/ftp/swisseph/ephe/semo_18.se1
!wget -q http://www.astro.com/ftp/swisseph/ephe/sepl_18.se1
!mkdir ephe
!mv *.se1 ephe
#------------------------------
# Configure SWE with appropriate parameters
#
p21swe.C01_configSWE()

##Read External Data
from CSV files


In [6]:
import pandas as pd

!wget -O peopleData.csv -q https://raw.githubusercontent.com/prithwis/parashar21/main/data/Test5Data.txt # 5 Person Test Data
#!wget -O peopleData.csv -q https://github.com/prithwis/parashar21/raw/main/data/Rodden39663.csv

#!gdown -O peopleData.csv https://drive.google.com/uc?id=1CVo1gnMZyCD0EZvAoF_wRdTwL7yqIT6L # 5 Person Test Data
#!gdown -O peopleData.csv https://drive.google.com/uc?id=1dlvqcygJJh0CfyY_X2MLrWm-W9U0z_6l # 39663 Rodden Data

#!sed -i '1d' peopleData.csv 
#!cat peopleData.csv

df = pd.read_csv('peopleData.csv')
df.head()

,Gender,Voc1_Cat,Voc1_Det,Voc2_Cat,Voc2_Det,Voc3_Cat,Voc3_Det,DoB_Day,DoB_Mon,DoB_Year,DoB_Time,TZ_Off,TZ_OffHours,PoB_Lat,PoB_Lon,TZ_Name,TZ_Type,Name
0,M,Engineer,nil,Teacher,nil,PostGraduate,nil,15,10,1961,23:42,h5e30,5.5,22.57,88.37,IST,standard,somu
1,F,Author,nil,Homemaker,nil,PostGraduate,nil,19,12,1965,10:26,h5e30,5.5,22.57,88.37,IST,standard,gopa
2,M,Engineer,nil,Teacher,nil,PostGraduate,nil,9,10,1990,23:09,h5e30,5.5,22.57,88.37,IST,standard,ahool
3,F,Biologist,nil,Scientist,nil,PostGraduate,nil,12,9,1990,0:52,h5e30,5.5,22.87,88.37,IST,standard,nilan
4,M,Politician,nil,Administrator,nil,nil,nil,17,9,1950,9:34,h5e30,5.5,23.48,72.36,IST,standard,NDModi


#Cast & Store charts in JSON file
Driver Code

In [7]:
%%time
p21.ChartType = 'Rashi'                                 # one of ['Rashi','Navamsa']

writeFile =  open("peopleData.json", "w")  
for person in range(len(df)):
    try :
        personData =df.iloc[person]
        p21swe.C02_parsePersonData(personData)                  # split the data, extract required data into fields
                                                                # create skeleton chart json doc as p21.chart
        p21swe.C03_convertDates()                               # convert date, time of birth into universal time, calculate Ayanamsha
        p21swe.C04_calculateGrahaPositions()                    # calculate the Lon of each Graha along with Retrograde staus

                    #Horoscope is defined in terms of Natal Longitudes and Retrogrades
                    # for example
                    #GLon = {"La":98.5,"Su":178.9,"Mo":250.6,"Ma":196.2,"Me":193.2,"Ju":274.8,"Ve":153.8,"Sa":270.2,"Ra":122.1,"Ke":302.1}
                    #GRet = {"La":False,"Su":False,"Mo":False,"Ma":False,"Me":True,"Ju":False,"Ve":False,"Sa":False,"Ra":True,"Ke":True}

        p21swe.C05_buildGLonGRet()                              # create two json docs ( python dicts) to store Graha Lon and Ret status
                                                                # these are stored as global variables p21.GLon, p21.GRet
        p21utils.appendDict(p21.chart,p21.GLonRet)              # add Lon, Ret information to chart
        
        # ----------------------------------------
        p21utils.R11_LocateGrahaInRashi()
        
        p21utils.C10_DetermineBhavs()                           # determine Houses for person
        p21utils.appendDict(p21.chart,p21.BhavNBhavA)           # add BhavN, BhavA information to chart   
         
        # ----------------------------------------    
        p21utils.C11_DetermineLord()                            # determine Lords of Bhavs
        p21utils.appendDict(p21.chart,p21.LordInfo)             #  add information on Lords   

        p21utils.C12_BhavOfGraha_Lord()
        p21utils.appendDict(p21.chart,p21.BhavOfGraha_LordInfo) #  add information on Bhav Residency of Graha and Lords  
        
        
        p21utils.C21_DeterminePositions()
        p21utils.appendDict(p21.chart,p21.Positions)            #  add information on Graha / Lord POSITIONS  

        p21utils.C31_DetermineAspects()
        p21utils.appendDict(p21.chart,p21.Aspects)              #  add information on Graha / Lord ASPECTS  

        p21utils.C41_DetermineConjuncts()           
        p21utils.appendDict(p21.chart,p21.Conjuncts)            #  add information on Graha / Lord Conjuncts
    except :
        print('error on record ', person)
    else :
        #insert_result = kollection.insert_one(p21.chart)        # insert one json doc, chart to database
        #insert_result.acknowledged                              # Confirms that insert is successful
                                                                 # Bypassing insertion into MongoDB will make processing much faster
        json.dump(p21.chart, writeFile)                          # Storing chart data as JSON file
    finally:
        if ( person % 1000 ) == 0:
            print(person, "records processed")
    
writeFile.close()
#!cp peopleData.json /content/drive/'My Drive'/Prithwis/Parashar21/ChartCast/ChartData2/

0 records processed
CPU times: user 11.8 ms, sys: 0 ns, total: 11.8 ms
Wall time: 15.3 ms


#Connect to MongoDB

In [8]:
from google.colab import drive
drive.mount('/content/drive')
# --------------------------------------------
# copy credentials file
#!cp /content/drive/'My Drive'/Praxis/WebCredentials/clevercloudMongoDB.py credentials.py
!cp /content/drive/'My Drive'/Praxis/WebCredentials/parashar21URI.py credentials.py
# -----------------------------------------------

#from credentials import rhostName,ruserName,rpassWord,rdbName,rMongoURI
from credentials import parashar21URI

# -----------------------------------------------
mongoURI = parashar21URI                                   # for remote server


Mounted at /content/drive


In [12]:
#mongoURI = "mongodb://localhost:27017/"
client = MongoClient(mongoURI)
client.list_database_names() 

['Parashar21', 'admin', 'local']

In [13]:
#db = client.btncgfi3a5bxyea
db = client.Parashar21
# Show Collections. Query returns a Cursor [ ] ,  list it to see the content
pd.DataFrame(list(db.list_collections()))
# empty list '[]' means that there are not collections in database

,name,type,options,info,idIndex
0,khona21Test,collection,{},"{'readOnly': False, 'uuid': b'\xce\xd6KOC:F\x9...","{'v': 2, 'key': {'_id': 1}, 'name': '_id_'}"


In [ ]:
#db = client.btncgfi3a5bxyea

#db.khona21Test.drop()                   # Test DB
#db.khona21.drop()                      # Main DB do not drop, accidentally

#db.create_collection('khona21Test')      # https://feminisminindia.com/2019/06/07/khawna-khana-poet-rural-bengal/
                                        # https://en.wikipedia.org/wiki/Khana_(poet)
#kollection = db.khona21Test                # Set the collection to work with

#Check and record Initial DB status
#countInitial = kollection.count_documents({})
#dbSizeInitial = db.command( {"dbStats": 1, "scale": 1024 } )['dataSize']
#print(round(dbSizeInitial),'KB database with ', countInitial, ' docs in current collection' )

#Load into Mongo

In [11]:
%%time
#!mongoimport --host "{rhostName}" --port "27017" --db "{rdbName}" --collection "khona21Test"  --drop --file "peopleData.json" -u "{ruserName}" -p "{rpassWord}"
#!mongoimport --uri "{rMongoURI}"  --collection "restaurants"  --drop --file "restaurants.json" 
#the original mongoURI needs to be changed to reflect the database where the collection will be loaded
mongoURIload = mongoURI.replace('?retryWrites=true&w=majority','Parashar21?')
##!mongoimport --uri "{rMongoURI}"  --collection "khona21Test"  --drop --file "peopleData.json" 
!mongoimport --uri "{mongoURIload}"  --collection "khona21Test"  --drop --file "peopleData.json" 

2022-06-13T04:54:00.245+0000	connected to: localhost
2022-06-13T04:54:00.459+0000	dropping: Parashar21.khona21Test
2022-06-13T04:54:01.694+0000	[########################] Parashar21.khona21Test	16.5KB/16.5KB (100.0%)
2022-06-13T04:54:02.279+0000	[########################] Parashar21.khona21Test	16.5KB/16.5KB (100.0%)
2022-06-13T04:54:02.279+0000	imported 5 documents
CPU times: user 50.7 ms, sys: 9.38 ms, total: 60.1 ms
Wall time: 3.73 s


#Check Database Status

In [15]:
#kollection = db.khona21
kollection = db.khona21Test
countFinal = kollection.count_documents({})
dbSizeFinal = db.command( {"dbStats": 1, "scale": 1024 } )['dataSize']
#DocsInserted = countFinal - countInitial
#AddSpaceUsed = round(dbSizeFinal - dbSizeInitial)
#if DocsInserted > 0:
#    AvgDocSize = round(AddSpaceUsed/DocsInserted)
#else:
#    AvgDocSize = 'na'
#print(DocsInserted,' docs inserted, ',AddSpaceUsed,' KB added , with avg doc size = ', AvgDocSize)
print(round(dbSizeFinal),'KB database with ', countFinal, ' docs in current collection' )

17 KB database with  5  docs in current collection


In [16]:
# Check data loaded into MongoDb
#c = kollection.find()
#for crnt in c:
#    print(crnt)
pd.DataFrame(list(kollection.find({},{'_id':0}).limit(10)))

,pid,GLon,GRet,BhavN,BhavA,Lord,LordRashiN,LordRashiA,GrahaLordBhav,GRashiN,...,ownHouseL,inFriendL,inEnemyL,GAspects2,GAspectedBy2,BAspectedBy2,BAspectedByBL2,GConjunctsG2,BLConjunctsG2,BLConjunctsBL2
0,"{'tags': ['Engineer', 'Teacher', 'PostGraduate...","{'La': 98.204, 'Su': 178.826, 'Mo': 250.52, 'M...","{'La': False, 'Su': False, 'Mo': False, 'Ma': ...","[ , 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3]","[ , Karkat, Simha, Kanya, Tula, Vrishchik, Dha...","[ , Mo, Su, Me, Ve, Ma, Ju, Sa, Sa, Ju, Ma, Ve...","[ , 9, 6, 7, 6, 7, 10, 10, 10, 10, 7, 6, 7]","[ , Dhanu, Kanya, Tula, Kanya, Tula, Makar, Ma...","{'Su': [2], 'Mo': [1], 'Ma': [5, 10], 'Me': [3...","{'La': 4, 'Su': 6, 'Mo': 9, 'Ma': 7, 'Me': 7, ...",...,"[False, False, False, False, False, False, Fal...","[False, False, False, True, True, False, False...","[False, False, False, False, False, False, Fal...","{'La': ['Ju', 'Sa'], 'Ma': ['Ju', 'Sa'], 'Ju':...","{'Ju': ['La', 'Ma'], 'Sa': ['La', 'Ma'], 'La':...","{'7': ['La', 'Ma'], '9': ['Su', 'Sa', 'Ve'], '...","{'1': ['6', '7', '9', '8'], '3': ['6', '9'], '...","{'Su': ['Ve'], 'Ve': ['Su'], 'Ma': ['Me'], 'Me...","{'2': ['Ve'], '3': ['Ma'], '4': ['Su'], '5': [...","{'2': ['11', '4'], '3': ['5', '10'], '4': ['2'..."
1,"{'tags': ['Author', 'Homemaker', 'PostGraduate...","{'La': 310.415, 'Su': 243.701, 'Mo': 201.001, ...","{'La': False, 'Su': False, 'Mo': False, 'Ma': ...","[ , 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[ , Kumbh, Meen, Mesh, Vrish, Mithun, Karkat, ...","[ , Sa, Ju, Ma, Ve, Me, Mo, Su, Me, Ve, Ma, Ju...","[ , 11, 3, 10, 10, 8, 7, 9, 8, 10, 10, 3, 11]","[ , Kumbh, Mithun, Makar, Makar, Vrishchik, Tu...","{'Su': [7], 'Mo': [6], 'Ma': [3, 10], 'Me': [5...","{'La': 11, 'Su': 9, 'Mo': 7, 'Ma': 10, 'Me': 8...",...,"[False, True, False, False, False, False, Fals...","[False, True, False, False, False, False, Fals...","[False, False, True, False, False, False, Fals...","{'Su': ['Ju'], 'Me': ['Ra'], 'Ju': ['Su', 'Mo'...","{'Ju': ['Su'], 'Ra': ['Me'], 'La': ['Ju'], 'Su...","{'7': ['La', 'Sa', 'Ma'], '5': ['Su'], '3': ['...","{'1': ['11', '2'], '3': ['12', '3', '10', '6',...","{'Ma': ['Ve'], 'Ve': ['Ma'], 'Me': ['Ke'], 'Ke...","{'3': ['Ve'], '4': ['Ma'], '5': ['Ke'], '8': [...","{'3': ['9', '4'], '4': ['3', '10'], '9': ['3',..."
2,"{'tags': ['Biologist', 'Scientist', 'PostGradu...","{'La': 84.138, 'Su': 145.068, 'Mo': 54.233, 'M...","{'La': False, 'Su': False, 'Mo': False, 'Ma': ...","[ , 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2]","[ , Mithun, Karkat, Simha, Kanya, Tula, Vrishc...","[ , Me, Mo, Su, Me, Ve, Ma, Ju, Sa, Sa, Ju, Ma...","[ , 5, 2, 5, 5, 5, 2, 4, 9, 9, 4, 2, 5]","[ , Simha, Vrish, Simha, Simha, Simha, Vrish, ...","{'Su': [3], 'Mo': [2], 'Ma': [6, 11], 'Me': [1...","{'La': 3, 'Su': 5, 'Mo': 2, 'Ma': 2, 'Me': 5, ...",...,"[False, False, False, True, False, False, Fals...","[False, True, False, False, True, False, False...","[False, False, False, False, False, True, Fals...","{'La': ['Sa'], 'Ma': ['Su', 'Sa', 'Ve', 'Me'],...","{'Sa': ['La', 'Ma'], 'Su': ['Ma'], 'Me': ['Ma'...","{'7': ['La', 'Ma'], '9': ['Su', 'Sa', 'Ve', 'M...","{'1': ['9', '8'], '3': ['6', '11'], '4': ['9',...","{'Su': ['Ve', 'Me'], 'Me': ['Su', 'Ve'], 'Ve':...","{'1': ['Su', 'Ve'], '2': ['Ma'], '3': ['Ve', '...","{'1': ['12', '3', '5'], '2': ['6', '11'], '3':..."
3,"{'tags': ['Engineer', 'Teacher', 'PostGraduate...","{'La': 85.351, 'Su': 172.44, 'Mo': 63.984, 'Ma...","{'La': False, 'Su': False, 'Mo': False, 'Ma': ...","[ , 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2]","[ , Mithun, Karkat, Simha, Kanya, Tula, Vrishc...","[ , Me, Mo, Su, Me, Ve, Ma, Ju, Sa, Sa, Ju, Ma...","[ , 6, 3, 6, 6, 6, 2, 4, 9, 9, 4, 2, 6]","[ , Kanya, Mithun, Kanya, Kanya, Kanya, Vrish,...","{'Su': [3], 'Mo': [2], 'Ma': [6, 11], 'Me': [1...","{'La': 3, 'Su': 6, 'Mo': 3, 'Ma': 2, 'Me': 6, ...",...,"[False, True, False, False, True, False, False...","[False, False, True, False, False, True, False...","[False, False, False, False, False, False, Fal...","{'La': ['Sa'], 'Mo': ['Sa'], 'Ma': 

In [17]:
#Retrieves Charts based on tags
pd.DataFrame(list(kollection.find({'pid.tags': {"$in": ['Scientist','Author']}},{'pid.tags':1,'GLon':1,'GRet':1,'_id':0})))
c = kollection.find({'pid.tags': {"$in": ['Scientist','Author']}},{'pid.tags':1,'GLon':1,'GRet':1,'_id':0})
for crnt in c:
    print(crnt)

{'pid': {'tags': ['Author', 'Homemaker', 'PostGraduate']}, 'GLon': {'La': 310.415, 'Su': 243.701, 'Mo': 201.001, 'Ma': 273.467, 'Me': 222.269, 'Ju': 62.748, 'Ve': 284.966, 'Sa': 318.17, 'Ra': 41.248, 'Ke': 221.248}, 'GRet': {'La': False, 'Su': False, 'Mo': False, 'Ma': False, 'Me': False, 'Ju': True, 'Ve': False, 'Sa': False, 'Ra': False, 'Ke': False}}
{'pid': {'tags': ['Biologist', 'Scientist', 'PostGraduate']}, 'GLon': {'La': 84.138, 'Su': 145.068, 'Mo': 54.233, 'Ma': 41.525, 'Me': 138.242, 'Ju': 101.216, 'Ve': 131.788, 'Sa': 265.084, 'Ra': 282.763, 'Ke': 102.763}, 'GRet': {'La': False, 'Su': False, 'Mo': False, 'Ma': False, 'Me': True, 'Ju': False, 'Ve': False, 'Sa': True, 'Ra': False, 'Ke': False}}


#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself. 
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)